# Test

- does the environment work?
- are packages imported?
- can we access the data?
- how is the data structured?
- how to import files in an efficient way?
- how to save numpy arrays?
- how to access saved numpy arrays?

In [2]:
import xarray as xr
import pop_tools
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

print('yay all installed <3')

yay all installed <3


In [ ]:
filename = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/bsf/bsf_1001.001.nc'
ds_bsf = xr.open_dataset(filename)

print('look at my pretty data')

ds_bsf

In [7]:
time = ds_bsf.time.values

print(time)

[cftime.DatetimeNoLeap(1850, 2, 1, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 3, 1, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1850, 4, 1, 0, 0, 0, 0, has_year_zero=True) ...
 cftime.DatetimeNoLeap(2100, 11, 1, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2100, 12, 1, 0, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2101, 1, 1, 0, 0, 0, 0, has_year_zero=True)]


In [3]:
path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc'
files = glob.glob(path + '/*.nc')

# loop through list of files
for f in files:
    
    # print files names
    print(f)
    
    # read in nc files
    #ds_moc = xr.open_dataset(f)

/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1231.019.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1301.017.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1231.012.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1251.007.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1231.017.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1301.016.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1301.006.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1251.003.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1281.010.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1231.013.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1281.013.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1281.007.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1301.015.nc
/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/moc_1251.004.nc
/Data/gfi/share/ModD

In [44]:
### INITIALISATION ###

import xarray as xr         # data handling
import glob                 # return all file paths that match a specific pattern
import ruptures as rpt      # for off-line change point detection

path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/moc/'
files = glob.glob(path + '/*.nc')

# length of time series
len_time = 3012

# prepare arrays and dictionaries to store files
time_series = np.zeros((len_time, len(files)))
arrays_dict = {} # Create a dictionary to store the arrays

### COMPUTATION ###

# loop through list of files
for i in range(2):
    # read in files
    ds_moc = xr.open_dataset(files[i])
    
    # compute maximum moc time series
    ds_max = ds_moc.MOC.isel(transport_reg=1, 
                             lat_aux_grid=slice(259,365), 
                             moc_z=slice(27, 51)
                            ).sum('moc_comp').max(('lat_aux_grid','moc_z'))
    # return numpy series
    signal = ds_max.values
    
    # compute change points
    algo_window = rpt.Window(model="rbf").fit(signal) 
    result = algo_window.fit_predict(signal, pen=40)
    
    # save signal to time series array
    time_series[:,i] = signal

    # Add the arra to the dictionary with a unique key
    key = f"array_{i}"
    arrays_dict[key] = result
    
### OUTPUT ###
    
# Save time series to a single file
np.save("moc_time_series.npy", time_series)
# Save all change point arrays containing to a single file
np.savez("moc_change_points.npz", **arrays_dict)

In [3]:
# Load the array from the file
data = np.load("moc_time_series.npy")
# Access the array
data[:,0]

array([24.7646656 , 25.51775932, 29.96545982, ..., 10.13521862,
       11.90265274,  9.95957565])

In [5]:
# Load all the arrays from the file
data = np.load("moc_change_points.npz")

for i in range(10):
    key = f"array_{i}"
    array = data[key]
    
    # Do something with the array
    print(f"Array {i} has shape {array.shape}")

Array 0 has shape (4,)
Array 1 has shape (6,)
Array 2 has shape (2,)
Array 3 has shape (2,)
Array 4 has shape (3,)
Array 5 has shape (3,)
Array 6 has shape (3,)
Array 7 has shape (4,)
Array 8 has shape (4,)
Array 9 has shape (4,)


In [3]:

import xarray as xr         # data handling
import glob                 # return all file paths that match a specific pattern

path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/bsf/'
files = glob.glob(path + '*.nc')
for i in range(len(files)):
    
    # read in files
    ds = xr.open_dataset(files[i])
    
    print(ds.BSF)

<xarray.DataArray 'BSF' (time: 3012, nlat: 384, nlon: 320)>
[370114560 values with dtype=float32]
Coordinates:
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    TLAT     (nlat, nlon) float64 ...
  * time     (time) object 1850-02-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Diagnostic barotropic streamfunction
    units:         Sv
    grid_loc:      2220
    cell_methods:  time: mean
<xarray.DataArray 'BSF' (time: 3012, nlat: 384, nlon: 320)>
[370114560 values with dtype=float32]
Coordinates:
    ULONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    TLAT     (nlat, nlon) float64 ...
  * time     (time) object 1850-02-01 00:00:00 ... 2101-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Attributes:
    long_name:     Diagnostic barotropic streamfunction
    units:         Sv
    grid_loc

In [3]:
path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/vvel/'
salt_path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/salt/'
temp_path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/temp/'
files = glob.glob(path + '*.nc')

In [11]:
glob.glob(salt_path + '*' + files[0][-11:])

['/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/salt/salt_1301.019.nc']

'1301.019.nc'